In [1]:
mindate='2020-04-01' #これより後の業績を集める
maxdate='2021-03-31' #これより前の業績を集める
allenglish = False #名前表記をすべて英語で統一する場合はTrue, 論文以外の名前表記を日本語にする場合False
sheeturl='https://docs.google.com/spreadsheets/d/1hf2oZbtyu-jCxEiljA8KiUVLZvIybFf0BrfnLc4nuYA/edit?usp=sharing' # 個人
#sheeturl='https://docs.google.com/spreadsheets/d/1P6SNqbCiIRCc5MNhZaK88fHui6Te6YPzrPADpg4Bgcw/edit?usp=sharing' # A1班
#sheeturl='https://docs.google.com/spreadsheets/d/1T7o73BqfOLZXCGTcFPvfZkZ916u9XbOLxINQbOGsfe8/edit?usp=sharing' # 総括班

setname='kojin'
yearname=mindate[:4]

papfilename='papers-'+yearname+'-'+setname+'.csv'
talkfilename='talks-'+yearname+'-'+setname+'.csv'
bookfilename='books-'+yearname+'-'+setname+'.csv'

In [2]:
import requests,json,sys,os,time,re,csv
import numpy as np
import pandas as pd

if 'google.colab' in str(get_ipython()):
    %pip install python-docx
    from google.colab import files,auth
    from oauth2client.client import GoogleCredentials
    outputdirectory = ''
else:
    outputdirectory = '../docx-researchmap-outputs/' #ローカルで実行する場合は保存ファイルのディレクトリを適当に指定
    os.makedirs(outputdirectory,exist_ok=True)

file_name_papers = outputdirectory+papfilename
file_name_talks = outputdirectory+talkfilename
file_name_books = outputdirectory+bookfilename

In [3]:
#スプレッドシートをダウンロード
sheeturl_csv=re.match("https://docs.google.com/spreadsheets/d/.+/",sheeturl).group(0)+"export?format=csv"
name_data=pd.read_csv(sheeturl_csv)

In [4]:
membernum=len(name_data)

allnames=(name_data["Surname"]+' '+name_data["First name"]).to_list()
allSurname=name_data["Surname"].to_list()
allnamesJP=(name_data["苗字"]+" "+name_data["名"]).to_list()
allmembers=name_data["researchmapID"].to_list()

In [5]:
def ReturnDictWOerror(dictdata,key,nodata):
    if key in dictdata.keys():
        return dictdata[key]
    else:
        return nodata

def ReturnDictContent(dictdata,key,key1,nodata=''):
    d=ReturnDictWOerror(dictdata,key,nodata)
    d1=ReturnDictWOerror(dictdata,key1,nodata)
    if d!=nodata:
        return d
    else:
        return d1

In [6]:
# Function to set the name order (surname-firstname)
def SurnameFirst(namesDic,sn):
    oldnamelist=[]
    swap=0
    for indiv in namesDic:
        oldnamelist=oldnamelist+[indiv['name'].replace(',','').replace('.','')]
        #print(oldnamelist)
    for name in oldnamelist:
        if sn in name.split(' '):
            if name.split(' ').index(sn)==0:
                swap=0
                break;
            else:
                swap=1
                break;
    if swap:
        newnamelist=[]
        for name in oldnamelist:
            namesplit=name.split(' ')
            names=[namesplit[-1]]+namesplit[:-1]
            newnamelist=newnamelist+[' '.join(names)]
    else:
        newnamelist=oldnamelist
    return newnamelist

In [7]:
#download json files from researchmap
url = "https://api.researchmap.jp/"
itemslist = ["published_papers","research_projects","misc","presentations","books_etc"]
jsonfiles={}
for name in allmembers:
  jsonfiles[name]={}
  for it in itemslist:
    r1 = requests.get(url+name+'/'+it)
    jsonfiles[name][it]=json.loads(r1.text)

In [8]:
# make dictionary of all papers
i=0
PapersDict={}

for ids,fullname in zip(allmembers,allnames):
    surname=fullname.split(' ')[0]
    dfP = jsonfiles[ids]["published_papers"]
    dfG = jsonfiles[ids]["research_projects"]
    if 'items' in dfP.keys():    
        for dfs in dfP['items']:
            if "authors" not in dfs.keys():
                continue
            if 'identifiers' in dfs.keys():
                doinum=[0]
                if 'doi' in dfs['identifiers'].keys():
                    doinum=dfs['identifiers']['doi']
                PapersDict[i]={}
                correspo=False
                if "published_paper_owner_roles" in dfs.keys():
                    if "corresponding" in dfs["published_paper_owner_roles"]:
                        correspo=True     
                if "publication_name" in dfs.keys():
                    if 'en' in dfs["publication_name"].keys():
                        jname=dfs["publication_name"]['en'].upper()
                        Sname=SurnameFirst(dfs["authors"]['en'],surname)
                    else:
                        jname=dfs["publication_name"]['ja'].upper()
                        Sname=SurnameFirst(dfs["authors"]['ja'],surname)
                else:
                    if 'en' in dfs["paper_title"].keys():
                        jname=dfs["paper_title"]['en'].upper()
                        Sname=SurnameFirst(dfs["authors"]['en'],surname)
                    else:
                        jname=dfs["paper_title"]['ja'].upper()
                        Sname=SurnameFirst(dfs["authors"]['ja'],surname)
                spage=''  
                if "starting_page" in dfs.keys():
                    spage=dfs["starting_page"]
                PapersDict[i]['starting_page']=spage
                if "ending_page" in dfs.keys():
                    PapersDict[i]['ending_page']=dfs["ending_page"]
                vol=''
                if "volume" in dfs.keys():
                    vol=dfs["volume"]
                PapersDict[i]['volume']=vol
                
                PapersDict[i]['paper_title']=ReturnDictContent(dfs['paper_title'],'en','ja','')
                PapersDict[i]['jname']=jname
                PapersDict[i]['researcher']=fullname
                PapersDict[i]['authors']=Sname
                PapersDict[i]['date']=dfs["publication_date"]
                PapersDict[i]['referee']=ReturnDictContent(dfs,'referee','referee',False)
                PapersDict[i]['doi']=doinum[0]
                PapersDict[i]['Corresp']=correspo
                i=i+1

In [9]:
# make dictionary of all talks
TalksDict={}
i=0
for ids,fullname,fullnameJP in zip(allmembers,allnames,allnamesJP):
    dfPr = jsonfiles[ids]["presentations"]
    if 'items' in dfPr.keys():
        for dfs in dfPr['items']:
            if all([a in dfs.keys() for a in ['invited',"presentation_title","event",'publication_date',"presenters"]]):
                if dfs['invited']:
                    if ('ja' in dfs["presenters"].keys()):
                        pname=dfs["presenters"]["ja"][0]["name"]
                    else:
                        pname=dfs["presenters"]["en"][0]["name"]
                    if ('ja' in dfs["event"].keys()):
                        ename=dfs["event"]["ja"]
                    else:
                        ename=dfs["event"]["en"]
                    if ('ja' in dfs["presentation_title"].keys()):
                        ptitle=dfs["presentation_title"]["ja"]
                    else:
                        ptitle=dfs["presentation_title"]["en"]
                    pdate=dfs["publication_date"]
                    TalksDict[i]={}
                    TalksDict[i]["presenter"]=fullname
                    if allenglish:
                      TalksDict[i]['printname']=fullname
                    else:
                      TalksDict[i]["printname"]=fullnameJP
                    TalksDict[i]["event"]=ename
                    TalksDict[i]["presentation_title"]=ptitle
                    TalksDict[i]["date"]=pdate
                    i=i+1

In [10]:
# make dictionary of all books_etc
booksDict={}
i=0
for ids,fullname,fullnameJP in zip(allmembers,allnames,allnamesJP):
  dfM = jsonfiles[ids]["books_etc"]
  if 'items' in dfM.keys():
    for dfs in dfM['items']:
      if all([a in dfs.keys() for a in ['authors',"book_title","publication_date"]]):
        if ('ja' in dfs["authors"].keys()):
            pname=dfs["authors"]["ja"][0]["name"]
        else:
            pname=dfs["authors"]["en"][0]["name"]
        if ('ja' in dfs["book_title"].keys()):
            ename=dfs["book_title"]["ja"]
        else:
            ename=dfs["book_title"]["en"]
        if "book_owner_range" in dfs.keys():
          if ('ja' in dfs["book_owner_range"].keys()):
            eoname=" \'"+dfs["book_owner_range"]["ja"]+"\',"
          else:
            eoname=" \'"+dfs["book_owner_range"]["en"]+"\',"
        else:
          eoname=''
        if "book_owner_role" in dfs.keys():
          brole=" ("+dfs["book_owner_role"]+"),"
        else:
          brole=','
        if "publisher" in dfs.keys():
          if ('ja' in dfs["publisher"].keys()):
            pub=" "+dfs["publisher"]["ja"]+","
          else:
            pub=" "+dfs["publisher"]["en"]+","
        else:
          pub=''
        pdate=dfs["publication_date"]
        #print(pname,ename,pdate)
        booksDict[i]={}
        booksDict[i]['authors']=fullname
        if allenglish:
          booksDict[i]['printname']=fullname
        else:
          booksDict[i]['printname']=fullnameJP
        if 'identifiers' not in dfs.keys():
          booksDict[i]["isbn"]=''
        else:
          if 'isbn' not in dfs["identifiers"].keys():
            booksDict[i]["isbn"]=''
          else:
            booksDict[i]["isbn"]=dfs["identifiers"]['isbn'][0]
            
        booksDict[i]["book_title"]=' \"'+ename+'\",'
        booksDict[i]["book_owner_role"]=brole
        booksDict[i]["book_owner_range"]=eoname
        booksDict[i]["publisher"]=pub
        booksDict[i]["date"]=pdate
        i=i+1

In [11]:
#論文
#DOI, 著者名, 論文表題, 雑誌名, 巻, 発行年, 最初と最後の頁, 査読の有無 (0 or 1), 国際共著 (0 or 1), オープンアクセス (0 or 1)
PapersDictSelected={k:PapersDict[k] for k in range(len(PapersDict)) if (PapersDict[k]['date']>mindate) & (PapersDict[k]['date']<maxdate)  & (PapersDict[k]['referee'])}
keys=list(PapersDictSelected.keys())
datelist=[PapersDictSelected[r]['date'] for r in keys]
arg=np.argsort(datelist)[::-1]
f = open(file_name_papers, 'w',encoding='utf-8')
writer = csv.writer(f, lineterminator='\n')
for r in arg:
    pap=PapersDictSelected[keys[r]]
    pages=pap['starting_page']
    authortext=''
    for pp in pap['authors']:
        authortext=authortext+pp+', '
    authortext=authortext[:-2]
    if 'ending_page' in pap.keys():
        pages=pages+'--'+pap['ending_page']
    refint=0
    if pap['referee']:
        refint=1
    #paperslist.append([pap['doi'],pap['authors'],pap['paper_title'],pap['jname'],pap['volume'],pap['date'][:4],pages,str(refint),str(0),str(0)])
    lists=[pap['doi'],authortext,pap['paper_title'],pap['jname'],pap['volume'],pap['date'][:4],pages,str(refint),str(0),str(0)]
    writer.writerow(lists)
f.close()


#学会発表
#発表者名, 発表表題, 学会等名, 発表年, 発表年, 招待講演 (0 or 1), 国際学会 (0 or 1)
TalksDictSelected={k:TalksDict[k] for k in range(len(TalksDict)) if (TalksDict[k]['date']>mindate) & (TalksDict[k]['date']<maxdate) }
keys=list(TalksDictSelected.keys())
datelist=[TalksDictSelected[r]['date'] for r in keys]
arg=np.argsort(datelist)[::-1]
f = open(file_name_talks, 'w',encoding='shift_jis')
writer = csv.writer(f, lineterminator='\n')
for r in arg:
    pap=TalksDictSelected[keys[r]]
    #print(lists)
    lists=[pap['printname'],pap['presentation_title'],pap['event'],pap['date'][:4],pap['date'][:4],str(1),str(0)]
    lists=[text.replace('\uff0d', '-') for text in lists]
    writer.writerow(lists)
f.close()


#著書
#ISBN, 著者名, 出版社, 書名, 発行年, 総ページ数
booksDictSelected={k:booksDict[k] for k in range(len(booksDict)) if (booksDict[k]['date']>mindate) & (booksDict[k]['date']<maxdate) }
keys=list(booksDictSelected.keys())
datelist=[booksDictSelected[r]['date'] for r in keys]
arg=np.argsort(datelist)[::-1]
f = open(file_name_books, 'w',encoding='shift_jis')
writer = csv.writer(f, lineterminator='\n')
for r in arg:
    pap=booksDictSelected[keys[r]]
    
    lists=[pap['isbn'],pap['printname'],pap["publisher"][:-1],pap["book_title"][:-1],pap['date'][:4],str(0)]
    #print(lists)
    writer.writerow(lists)
f.close()

In [12]:
if 'google.colab' in str(get_ipython()):
    files.download(file_name_papers)
    files.download(file_name_talks)
    files.download(file_name_books)